In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
import time
import os

In [2]:
# Generate new data, if needed

X_baseline = [] # baseline time point
for filename in os.listdir("ADNIMERGE/baseline/"):
    ct = np.loadtxt("ADNIMERGE/baseline/"+filename, delimiter='\n')
    X_baseline.append(ct)
    
X_followup = [] # other longitudinal time point
for filename in os.listdir("ADNIMERGE/followup/"):
    ct = np.loadtxt("ADNIMERGE/followup/"+filename, delimiter='\n')
    X_followup.append(ct)
    
np.save("X_baseline.npy", np.array(X_baseline))
np.save("X_followup.npy", np.array(X_followup))

In [2]:
# Load the data

sub_list = pd.read_csv("Exp_CL1_sub_list.csv")
df = pd.read_csv("Exp_502_602_combined.csv")
X_baseline = np.load("X_baseline.npy")
X_followup = np.load("X_followup.npy")
X_diff = X_followup - X_baseline # vertex-wise CT change
n_subjects = sub_list.shape[0]
print(X_baseline.shape)
print(X_followup.shape)

(1061, 81924)
(1061, 81924)


In [6]:
# Generate the train-test splits

frac_train = 0.8
ncv = 10
splits = {"train": [], "test": []}

for i in range(ncv):
    indices = list(range(n_subjects))
    random.shuffle(indices)
    train_split = indices[:int(0.8*len(indices))]
    valid_split = indices[int(0.8*len(indices)):]
    splits["train"].append(train_split)
    splits["test"].append(valid_split)
    
splits = pd.DataFrame.from_dict(splits)
splits.to_pickle("train_test_splits.pkl")

In [7]:
# Reduce the data with PCA
from sklearn.decomposition import PCA

n_components = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

for i in range(ncv):
    train_split = splits["train"][i]
    test_split = splits["test"][i]
    X_train_baseline = X_baseline[train_split]
    X_test_baseline = X_baseline[test_split]
    X_train_followup = X_followup[train_split]
    X_test_followup = X_followup[test_split]
    X_train_diff = X_diff[train_split]
    
    tstart = time.time()
    pca = PCA(n_components=n_components)
    pca.fit(X_train_diff)
    X_bl_train_reduced = pca.transform(X_train_baseline)
    X_bl_test_reduced = pca.transform(X_test_baseline)
    X_vartp_train_reduced = pca.transform(X_train_followup)
    X_vartp_test_reduced = pca.transform(X_test_followup)
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/PCA_bl_train_cv{}.npy".format(i), X_bl_train_reduced)
    np.save("data/PCA_bl_test_cv{}.npy".format(i), X_bl_test_reduced)
    np.save("data/PCA_vartp_train_cv{}.npy".format(i), X_vartp_train_reduced)
    np.save("data/PCA_vartp_test_cv{}.npy".format(i), X_vartp_test_reduced)

Time required : 25.57176661491394
Time required : 13.149729251861572
Time required : 14.0746169090271
Time required : 14.54721474647522
Time required : 14.454704284667969
Time required : 14.827574253082275
Time required : 14.644176959991455
Time required : 14.727607488632202
Time required : 14.855762481689453
Time required : 14.59036636352539


In [8]:
# Reduce the data with RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

n_features = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

for i in range(ncv):
    train_split = splits["train"][i]
    test_split = splits["test"][i]
    X_train_baseline = X_baseline[train_split]
    X_test_baseline = X_baseline[test_split]
    X_train_followup = X_followup[train_split]
    X_test_followup = X_followup[test_split]
    X_train_diff = X_diff[train_split]
    
    ptid = sub_list["PTID"]
    train_ptid = ptid[train_split]
    df_train = df[df["PTID"].isin(train_ptid)]
    y_train_mmse = df_train["MMSE_2c_traj"]
    y_train_adas = df_train["ADAS_3c_traj"]
    
    tstart = time.time()
    estimator = SVC(kernel="linear")
    rfe = RFE(estimator, n_features, step=0.5)
    rfe.fit(X_train_diff, y_train_mmse)
    X_bl_train_mmse = rfe.transform(X_train_baseline)
    X_bl_test_mmse = rfe.transform(X_test_baseline)
    X_vartp_train_mmse = rfe.transform(X_train_followup)
    X_vartp_test_mmse = rfe.transform(X_test_followup)
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/RFE_bl_train_MMSE_cv{}.npy".format(i), X_bl_train_mmse)
    np.save("data/RFE_bl_test_MMSE_cv{}.npy".format(i), X_bl_test_mmse)
    np.save("data/RFE_vartp_train_MMSE_cv{}.npy".format(i), X_vartp_train_mmse)
    np.save("data/RFE_vartp_test_MMSE_cv{}.npy".format(i), X_vartp_test_mmse)
    
    tstart = time.time()
    estimator = SVC(kernel="linear")
    rfe = RFE(estimator, n_features, step=0.5)
    rfe.fit(X_train_diff, y_train_adas)
    X_bl_train_adas = rfe.transform(X_train_baseline)
    X_bl_test_adas = rfe.transform(X_test_baseline)
    X_vartp_train_adas = rfe.transform(X_train_followup)
    X_vartp_test_adas = rfe.transform(X_test_followup)
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/RFE_bl_train_ADAS13_cv{}.npy".format(i), X_bl_train_adas)
    np.save("data/RFE_bl_test_ADAS13_cv{}.npy".format(i), X_bl_test_adas)
    np.save("data/RFE_vartp_train_ADAS13_cv{}.npy".format(i), X_vartp_train_adas)
    np.save("data/RFE_vartp_test_ADAS13_cv{}.npy".format(i), X_vartp_test_adas)

Time required : 126.70930600166321
Time required : 175.61174726486206
Time required : 124.69630217552185
Time required : 174.79371762275696
Time required : 123.47172927856445
Time required : 180.70509338378906
Time required : 129.811341047287
Time required : 167.86436414718628
Time required : 122.67340397834778
Time required : 171.08620715141296
Time required : 130.93012809753418
Time required : 173.38778924942017
Time required : 123.8691246509552
Time required : 172.9858980178833
Time required : 125.03823852539062
Time required : 183.5230188369751
Time required : 137.060405254364
Time required : 168.04962372779846
Time required : 122.51985120773315
Time required : 168.7333207130432


In [40]:
# Reduce the data with RLR
# Idea by Moradi et al., 2015
from sklearn.linear_model import SGDClassifier

n_features = 78 # same number of components as AAL
splits = pd.read_pickle("train_test_splits.pkl")

n_repeats = 5 # 10 by Moradi et al.
range_size = 10 # 100 by Moradi et al.
alpha_range = np.power(10, np.linspace(-5, -2, range_size))
print("Range of alphas: {}".format(alpha_range))

for i in range(ncv):
    train_split = splits["train"][i]
    test_split = splits["test"][i]
    X_train_baseline = X_baseline[train_split]
    X_test_baseline = X_baseline[test_split]
    X_train_followup = X_followup[train_split]
    X_test_followup = X_followup[test_split]
    X_train_diff = X_diff[train_split]
    
    ptid = sub_list["PTID"]
    train_ptid = ptid[train_split]
    df_train = df[df["PTID"].isin(train_ptid)]
    y_train_mmse = df_train["MMSE_2c_traj"]
    y_train_adas = df_train["ADAS_3c_traj"]
    
    print("Begin MMSE, CV fold number {}".format(i))
    print("Step one: choose the optimal alpha")
    tstart = time.time()
    alpha_star = []
    
    for j in range(n_repeats):
        print("Beginning repeat {}...".format(j))
        scores = np.zeros(range_size)
        
        indices = list(range(len(train_split)))
        random.shuffle(indices)
        inner_train = indices[:int(0.9*len(indices))]
        inner_valid = indices[int(0.9*len(indices)):]
        X_diff_inner_train = X_diff[inner_train]
        X_diff_inner_valid = X_diff[inner_valid]
        inner_train_ptid = ptid[inner_train]
        inner_valid_ptid = ptid[inner_valid]
        df_inner_train = df[df["PTID"].isin(inner_train_ptid)]
        df_inner_valid = df[df["PTID"].isin(inner_valid_ptid)]
        y_inner_train_mmse = df_inner_train["MMSE_2c_traj"]
        y_inner_valid_mmse = df_inner_valid["MMSE_2c_traj"]
        
        for k in range(range_size):
            clf = SGDClassifier(loss="log", penalty="elasticnet", l1_ratio=0.5, alpha=alpha_range[k])
            clf.fit(X_diff_inner_train, y_inner_train_mmse)
            scores[k] = clf.score(X_diff_inner_valid, y_inner_valid_mmse)
        
        alpha_star.append(alpha_range[np.argmax(scores)])
        
    print("Optimal alpha: {}".format(np.median(alpha_star)))
    print("Step two: find the most significant features")
    k = np.where(alpha_range <= np.median(alpha_star))[0][-1]
    nonzero_features = np.ones(X_baseline.shape[1])
    
    for alpha in alpha_range[k:]:
        clf = SGDClassifier(loss="log", penalty="elasticnet",
                            l1_ratio=0.5, alpha=alpha)
        clf.fit(X_train_diff, y_train_mmse)
        nonzero_features = np.logical_and(nonzero_features, (clf.coef_ > 0))
    
    nonzero_indices = np.where(nonzero_features)[1]
    X_train_diff_remain = X_train_diff[:, nonzero_indices]
    print("Number of nonzero features: {}".format(nonzero_indices.shape[0]))
    coef_sums = np.zeros(X_train_diff_remain.shape[1])
    
    for j in range(n_repeats):
        clf = SGDClassifier(loss="log", penalty="l2", alpha=alpha_range[k])
        clf.fit(X_train_diff_remain, y_train_mmse)
        coef_sums = coef_sums + clf.coef_
        
    most_significant = np.argsort(coef_sums)[0, :78]
    features_selected = nonzero_indices[most_significant]
    
    X_bl_train_mmse = X_train_baseline[:, features_selected]
    X_bl_test_mmse = X_test_baseline[:, features_selected]
    X_vartp_train_mmse = X_train_followup[:, features_selected]
    X_vartp_test_mmse = X_test_followup[:, features_selected]
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/RLR_bl_train_MMSE_cv{}.npy".format(i), X_bl_train_mmse)
    np.save("data/RLR_bl_test_MMSE_cv{}.npy".format(i), X_bl_test_mmse)
    np.save("data/RLR_vartp_train_MMSE_cv{}.npy".format(i), X_vartp_train_mmse)
    np.save("data/RLR_vartp_test_MMSE_cv{}.npy".format(i), X_vartp_test_mmse)
    
    
    print("Begin ADAS13, CV fold number {}".format(i))
    print("Step one: choose the optimal alpha")
    tstart = time.time()
    alpha_star = []
    
    for j in range(n_repeats):
        print("Beginning repeat {}...".format(j))
        scores = np.zeros(range_size)
        
        indices = list(range(len(train_split)))
        random.shuffle(indices)
        inner_train = indices[:int(0.9*len(indices))]
        inner_valid = indices[int(0.9*len(indices)):]
        X_diff_inner_train = X_diff[inner_train]
        X_diff_inner_valid = X_diff[inner_valid]
        inner_train_ptid = ptid[inner_train]
        inner_valid_ptid = ptid[inner_valid]
        df_inner_train = df[df["PTID"].isin(inner_train_ptid)]
        df_inner_valid = df[df["PTID"].isin(inner_valid_ptid)]
        y_inner_train_adas = df_inner_train["ADAS_3c_traj"]
        y_inner_valid_adas = df_inner_valid["ADAS_3c_traj"]
        
        for k in range(range_size):
            clf = SGDClassifier(loss="log", penalty="elasticnet", l1_ratio=0.5, alpha=alpha_range[k])
            clf.fit(X_diff_inner_train, y_inner_train_adas)
            scores[k] = clf.score(X_diff_inner_valid, y_inner_valid_adas)
        
        alpha_star.append(alpha_range[np.argmax(scores)])
        
    print("Optimal alpha: {}".format(np.median(alpha_star)))
    print("Step two: find the nonzero features")
    k = np.where(alpha_range <= np.median(alpha_star))[0][-1]
    nonzero_features = np.ones(X_baseline.shape[1])
    
    for alpha in alpha_range[k:]:
        clf = SGDClassifier(loss="log", penalty="elasticnet",
                            l1_ratio=0.5, alpha=alpha)
        clf.fit(X_train_diff, y_train_adas)
        nonzero_features = np.logical_and(nonzero_features, (clf.coef_ > 0))
    
    nonzero_indices = np.where(nonzero_features)[1]
    X_train_diff_remain = X_train_diff[:, nonzero_indices]
    print("Number of nonzero features: {}".format(nonzero_indices.shape[0]))
    coef_sums = np.zeros(X_train_diff_remain.shape[1])
    
    for j in range(n_repeats):
        clf = SGDClassifier(loss="log", penalty="l2", alpha=alpha_range[k])
        clf.fit(X_train_diff_remain, y_train_adas)
        coef_sums = coef_sums + clf.coef_
        
    most_significant = np.argsort(coef_sums)[0, :78]
    features_selected = nonzero_indices[most_significant]
    
    X_bl_train_adas = X_train_baseline[:, features_selected]
    X_bl_test_adas = X_test_baseline[:, features_selected]
    X_vartp_train_adas = X_train_followup[:, features_selected]
    X_vartp_test_adas = X_test_followup[:, features_selected]
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/RLR_bl_train_ADAS13_cv{}.npy".format(i), X_bl_train_adas)
    np.save("data/RLR_bl_test_ADAS13_cv{}.npy".format(i), X_bl_test_adas)
    np.save("data/RLR_vartp_train_ADAS13_cv{}.npy".format(i), X_vartp_train_adas)
    np.save("data/RLR_vartp_test_ADAS13_cv{}.npy".format(i), X_vartp_test_adas)

Range of alphas: [1.00000000e-05 2.15443469e-05 4.64158883e-05 1.00000000e-04
 2.15443469e-04 4.64158883e-04 1.00000000e-03 2.15443469e-03
 4.64158883e-03 1.00000000e-02]
Begin MMSE, CV fold number 0
Step one: choose the optimal alpha
Beginning repeat 0...


/opt/quarantine/anaconda/5.1.0-python3/install/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 4.641588833612782e-05
Step two: find the most significant features
Number of nonzero features: 897
Time required : 251.42656755447388
Begin ADAS13, CV fold number 0
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 0.01
Step two: find the nonzero features
Number of nonzero features: 3981
Time required : 619.5794517993927
Begin MMSE, CV fold number 1
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning repeat 3...
Beginning repeat 4...
Optimal alpha: 0.002154434690031882
Step two: find the most significant features
Number of nonzero features: 767
Time required : 223.5866858959198
Begin ADAS13, CV fold number 1
Step one: choose the optimal alpha
Beginning repeat 0...
Beginning repeat 1...
Beginning repeat 2...
Beginning r

In [12]:
# Reduce the data with HCA
from sklearn.cluster import AgglomerativeClustering

n_features = 78 # same number of components as AAL
n_partitions = 20 # 2-step HCA process
partition_size = 40962/n_partitions
n_clusters = int(partition_size/n_partitions)
splits = pd.read_pickle("train_test_splits.pkl")

for i in range(ncv):
    train_split = splits["train"][i]
    test_split = splits["test"][i]
    X_train_baseline = X_baseline[train_split]
    X_test_baseline = X_baseline[test_split]
    X_train_followup = X_followup[train_split]
    X_test_followup = X_followup[test_split]
    X_train_diff = X_diff[train_split]
    
    def generate_new_features(X, left, right):
        new_features = np.zeros((X.shape[0], n_features))
        
        for cluster in np.unique(left_clustering.labels_):
            vertices = X[:, np.where(left_clustering.labels_ == cluster)[0]]
            new_features[:, cluster] = np.mean(vertices)
            
        for cluster in np.unique(right_clustering.labels_):
            vertices = X[:, np.where(right_clustering.labels_ == cluster)[0]]
            new_features[:, cluster + 39] = np.mean(vertices)
        
        return new_features
    
    tstart = time.time()
    print("Beginning HCA, CV fold number {}".format(i))
    print("Clustering left vertices...")
    
    left_vertices = X_train_diff.T[:40962, :]
    left_vertices_merged = []
    left_cluster_list = []
    
    for p in range(n_partitions):
        vp = left_vertices[int(p*partition_size):int((p+1)*partition_size)]
        hca = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
        clustering = hca.fit(vp)
        
        for cluster in np.unique(clustering.labels_):
            ind = np.where(clustering.labels_ == cluster)[0]
            left_cluster_list.append(ind + int(p*partition_size))
            left_vertices_merged.append(np.mean(vp[ind, :], axis=0))

    hca = AgglomerativeClustering(n_clusters=int(np.floor(n_features/2)), linkage="ward")
    left_clustering = hca.fit(np.array(left_vertices_merged))
        
    print("Clustering right vertices...")
    
    right_vertices = X_train_diff.T[-40962:, :]
    right_vertices_merged = []
    right_cluster_list = []
    
    for p in range(n_partitions):
        vp = right_vertices[int(p*partition_size):int((p+1)*partition_size)]
        hca = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
        clustering = hca.fit(vp)
        
        for cluster in np.unique(clustering.labels_):
            ind = np.where(clustering.labels_ == cluster)[0]
            right_cluster_list.append(ind + int(p*partition_size))
            right_vertices_merged.append(np.mean(vp[ind, :], axis=0))
            
    hca = AgglomerativeClustering(n_clusters=int(np.ceil(n_features/2)), linkage="ward")
    right_clustering = hca.fit(np.array(right_vertices_merged))
    
    print("Reducing the data...")
    
    X_bl_train_reduced = np.zeros((X_train_baseline.shape[0], n_features))
    X_bl_test_reduced = np.zeros((X_test_baseline.shape[0], n_features))
    X_vartp_train_reduced = np.zeros((X_train_followup.shape[0], n_features))
    X_vartp_test_reduced = np.zeros((X_test_followup.shape[0], n_features))
    
    for cluster in np.unique(left_clustering.labels_):
        subclusters = np.where(left_clustering.labels_ == cluster)[0]
        ind = np.concatenate([left_cluster_list[sc] for sc in subclusters])
        X_bl_train_reduced[:, cluster] = np.mean(X_train_baseline[:, ind], axis=1)
        X_bl_test_reduced[:, cluster] = np.mean(X_test_baseline[:, ind], axis=1)
        X_vartp_train_reduced[:, cluster] = np.mean(X_train_followup[:, ind], axis=1)
        X_vartp_test_reduced[:, cluster] = np.mean(X_test_followup[:, ind], axis=1)
        
    shift = int(np.floor(n_features/2))
    for cluster in np.unique(right_clustering.labels_):
        subclusters = np.where(right_clustering.labels_ == cluster)[0]
        ind = np.concatenate([right_cluster_list[sc] for sc in subclusters])
        X_bl_train_reduced[:, cluster + shift] = np.mean(X_train_baseline[:, ind], axis=1)
        X_bl_test_reduced[:, cluster + shift] = np.mean(X_test_baseline[:, ind], axis=1)
        X_vartp_train_reduced[:, cluster + shift] = np.mean(X_train_followup[:, ind], axis=1)
        X_vartp_test_reduced[:, cluster + shift] = np.mean(X_test_followup[:, ind], axis=1)
    
    print("Time required : {}".format(time.time() - tstart))
    np.save("data/HCA_bl_train_cv{}.npy".format(i), X_bl_train_reduced)
    np.save("data/HCA_bl_test_cv{}.npy".format(i), X_bl_test_reduced)
    np.save("data/HCA_vartp_train_cv{}.npy".format(i), X_vartp_train_reduced)
    np.save("data/HCA_vartp_test_cv{}.npy".format(i), X_vartp_test_reduced)

Beginning HCA, CV fold number 0
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 62.664196252822876
Beginning HCA, CV fold number 1
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 62.65158700942993
Beginning HCA, CV fold number 2
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 64.05795669555664
Beginning HCA, CV fold number 3
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 62.70123505592346
Beginning HCA, CV fold number 4
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 64.898353099823
Beginning HCA, CV fold number 5
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 64.6041955947876
Beginning HCA, CV fold number 6
Clustering left vertices...
Clustering right vertices...
Reducing the data...
Time required : 63.109287738